##  England Comparison of Hospital Admission with Ventilator bed Occupancy

In [ ]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Accessing PHE Covid Data

Data provided has been collected is from  Primary Health England(PHE) who are providing current statistics on the COVID-19

In [ ]:
with open("Eng_hospital_admissions.json", "rt") as INFILE:
    data=json.load(INFILE)

data

## Wrangling the Data

In [ ]:
datalist=data['data']
datalist

In [ ]:
dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()
dates

In [ ]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [ ]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
print (startdate, ' to ', enddate)

In [ ]:
index=pd.date_range(startdate, enddate, freq='D')
Eng_hospital_admissionsdf=pd.DataFrame(index=index, columns=['hospitalCases', 'covidOccupiedMVBeds'])
Eng_hospital_admissionsdf

In [ ]:
for entry in datalist: 
    date=parse_date(entry['date'])
    for column in [ 'hospitalCases', 'covidOccupiedMVBeds']:
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
        if pd.isna(Eng_hospital_admissionsdf.loc[date, column]): 
            # replace None with 0 in our data 
            value= float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            Eng_hospital_admissionsdf.loc[date, column]=value

            
# fill in any remaining "holes" due to missing dates
Eng_hospital_admissionsdf.fillna(0.0, inplace=True)
            

                
print(Eng_hospital_admissionsdf)

## Below is a graph comparing Hospital Cases with Ventilator bed Occupancy

In [ ]:
Eng_hospital_admissionsdf.plot()

In [ ]:
# our API access function. This will be called by the button when it is clicked
def access_api(button):
    # Ignore the parameter, put code for polling the API here
    print("I'm downloading data from the API...")
    print("...all done.")
    
# see the doc for the parameters    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)

## For Above, clicking the refresh data button, you will be able to see up to date statistics 

In [ ]:
Eng_hospital_admissionsdf=pd.read_pickle("Eng_hospital_admissionsdf.pkl")

In [ ]:
series=wdg.SelectMultiple(
    options=['hospitalCases', 'covidOccupiedMVBeds'],
    value=['hospitalCases',  'covidOccupiedMVBeds'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])

def Eng_hospital_admissions_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        Eng_hospital_admissionsdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(Eng_hospital_admissions_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)